In [1]:
import os
import sys

import numpy as np
import SimpleITK as sitk
import torch
import pandas as pd
import pytorch_lightning as pl
import torchvision

import matplotlib.pyplot as plt


# Move to top of ArtifactNet directory so relative imports can be used here
sys.path.insert(0, "/cluster/home/carrowsm/ArtifactNet")

# Plotting Sample Images
Use this notebook to load different models and pass sample images to them.

## Define functions to load the PyTorch-Lightning models and the dataloaders

In [2]:
def load_model(module: pl.LightningModule, checkpoint_path: str) :
    model = module.load_from_checkpoint(PATH)
    print(model.learning_rate)
    # prints the learning_rate you used in this checkpoint

    model.eval()
    return model

In [3]:
# Import dataloaders
from data.data_loader import PairedDataset, UnpairedDataset, load_image_data_frame
from data.transforms import AffineTransform, ToTensor, Normalize

# Load data frames with image labels
val_x_df = pd.read_csv("/cluster/home/carrowsm/ArtifactNet/datasets/reza_test_X.csv", dtype=str)
val_x_df.set_index("patient_id", inplace=True)
val_y_df = pd.read_csv("/cluster/home/carrowsm/ArtifactNet/datasets/reza_test_Y.csv", dtype=str)
val_y_df.set_index("patient_id", inplace=True)

csv_path = "/cluster/home/carrowsm/ArtifactNet/datasets/train_labels.csv"
y_df, n_df = load_image_data_frame(csv_path, ["2"], ["1"])

# Save paths to data
img_dir = "/cluster/projects/radiomics/RADCURE-images/"
cache_dir = "/cluster/projects/radiomics/Temp/colin/isotropic_nrrd/"

In [4]:
# Define sequence of transforms
trg_transform = torchvision.transforms.Compose([
                    AffineTransform(max_angle=20.0, max_pixels=[20, 20]), 
                    Normalize(-1000.0, 1000.0),
                    ToTensor()])
val_transform = torchvision.transforms.Compose([
                    ToTensor()])

# Initialize the two dataloaders
trg_dataset = UnpairedDataset(y_df, n_df,
                              image_dir=img_dir,
                              cache_dir=cache_dir,
                              file_type="DICOM",
                              image_size=[8, 256,256],
                              dim=3,
                              transform=trg_transform,
                              num_workers=5)
val_dataset = PairedDataset(val_x_df, val_x_df,
                            image_dir=img_dir,
                            cache_dir=os.path.join(cache_dir, "paired_test"),
                            file_type="DICOM",
                            image_size=[8, 256,256],
                            dim=3,
                            transform=val_transform,
                            num_workers=5)


Data successfully cached


Data successfully cached



In [7]:
val1X, val1Y = val_dataset[0]
val2X, val2Y = val_dataset[1]


tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-0.4528, -0.4530, -0.4589,  ..., -0.4596, -0.4588, -0.4515],
          [-0.4515, -0.4550, -0.4566,  ..., -0.4614, -0.4583, -0.4522],
          [-0.4585, -0.4577, -0.4543,  ..., -0.4580, -0.4533, -0.4554],
          ...,
          [-0.4572, -0.4553, -0.4514,  ..., -0.4514, -0.4545, -0.4548],
          [-0.4551, -0.4518, -0.4509,  ..., -0.4479, -0.4507, -0.4575],
          [-0.4474, -0.4489, -0.4493,  ..., -0.4473, -0.4483, -0.4518]],

         [[-0.4922, -0.4924, -0.4971,  ..., -0.4987, -0.4974, -0.4906],
          [-0.4901, -0.4936, -